In [1]:
%matplotlib inline
%run ../../import_envs.py
print('probtorch:', probtorch.__version__, 
      'torch:', torch.__version__, 
      'cuda:', torch.cuda.is_available())

probtorch: 0.0+5a2c637 torch: 0.4.1 cuda: True


In [2]:
## Load dataset
data_path = "../gmm_dataset_c5k"
Data = torch.from_numpy(np.load(data_path + '/obs.npy')).float()

NUM_DATASETS, N, D = Data.shape
K = 3 ## number of clusters
SAMPLE_SIZE = 10
NUM_HIDDEN_GLOBAL = 16
NUM_HIDDEN_LOCAL = 16
NUM_LAYERS = 1

BATCH_SIZE = 20
NUM_EPOCHS = 500
LEARNING_RATE = 5 * 1e-4
CUDA = torch.cuda.is_available()
PATH = 'baseline-lstm-cfz-%dsamples' % (SAMPLE_SIZE)
DEVICE = torch.device('cuda:0')

Train_Params = (NUM_EPOCHS, NUM_DATASETS, SAMPLE_SIZE, BATCH_SIZE, CUDA, DEVICE, PATH)
Model_Params = (N, K, D)

In [3]:
from global_lstm import *
from local_gibbs import *
## if reparameterize continuous variables
Reparameterized = True

lstm_eta = LSTM_eta(K, D, BATCH_SIZE, SAMPLE_SIZE, NUM_HIDDEN_GLOBAL, NUM_HIDDEN_LOCAL, CUDA, DEVICE, Reparameterized)
enc_z = Gibbs_z(K, CUDA, DEVICE)
if CUDA:
    lstm_eta.cuda().to(DEVICE)
optimizer =  torch.optim.Adam(list(lstm_eta.parameters()),lr=LEARNING_RATE, betas=(0.9, 0.99))

models = (lstm_eta, enc_z)

In [4]:
from os_ep import *
train(models, EP, optimizer, Data, Model_Params, Train_Params)

epoch: 0\500 (19s),  eubo: -2199.828,  elbo: -2861.833,  ess: 1.010,  kl_eta_ex: 3157.888,  kl_eta_in: 214.085,  kl_z_ex: 0.000,  kl_z_in: -0.000


KeyboardInterrupt: 

In [ ]:
torch.save(enc_eta.state_dict(), '../weights/enc-eta-%s' + PATH)
torch.save(oneshot_eta.state_dict(), '../weights/oneshot-eta-%s' + PATH)

In [ ]:
BATCH_SIZE_TEST = 50
Model_Params_Test = (N, K, D, MCMC_SIZE, SAMPLE_SIZE, BATCH_SIZE_TEST)
obs, metric_step, reused = test(Eubo_cfz_os_eta, Data, Train_Params, Model_Params_Test, models)
(q_eta, _, q_z, _, _, _) = reused

In [ ]:
%time plot_samples(obs, q_eta, q_z, PATH)

In [ ]:
incremental_gap = symkl_test.cpu().data.numpy()[1:]
M = incremental_gap.shape[0]
overall_gap = np.zeros(M)
for m in range(M):
    overall_gap[m] = incremental_gap[:m+1].sum()

In [ ]:
fig = plt.figure(figsize=(12,6))
ax = fig.add_subplot(111)
plt.yscale("log")
ax.plot(incremental_gap, label="incremental gap")
ax.plot(overall_gap, label='overall gap')
ax.legend(fontsize=14)
ax.set_xlabel('Steps')

In [ ]:
incremental_gap